In [1]:
from itertools import product
from math import sin, cos
from tqdm import tqdm
import numpy as np
from z3 import *

### Constantes globais

In [2]:
V_LOW, V_HIGH = 1, 10 # velocidades dos modos baixo e alto, em m
THETA = 15 # angulo de viragem entre modos
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis

GAUSS_DP = 80 # desvio padrão da gaussiana de inicialização das posições em m
SAFE_DIST = 50 # distância de segurança entre barcos em m

### Funções auxiliares

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
rad_to_deg = lambda a: a * 180 / np.pi
z3tofloat = lambda v: float(v.numerator_as_long())/float(v.denominator_as_long())
Abs = lambda x: If(x>=0, x, -x)
val_angle = lambda a: If(a<0, 360+a, If(a>=360, a-360, a))

## Funções de declaração, inicialização e transição

In [6]:
def declare(t, num_boats=3):
    # Iterar os barcos
    trace = {}
    for i in range(num_boats):
        trace[i] = {}
        
        # Variáveis de modo
        trace[i]["v"] = Int(f"b{i}_v{t}")
        trace[i]["a"] = Int(f"b{i}_a{t}")
        
        # Variáveis de estado
        trace[i]["x"] = Real(f"b{i}_x{t}")
        trace[i]["y"] = Real(f"b{i}_y{t}")
        trace[i]["t"] = Real(f"b{i}_t{t}")
        
    return trace
        
        
def init(tr):
    # Gerar ângulos iniciais aleatórios
    random_angles = ALL_ROUTES
    
    # Verificar que há mais barcos que ângulos possíveis
    assert len(tr) <= len(random_angles)
    
    # Iterar os barcos
    r = []
    for i in range(len(tr)):
        np.random.shuffle(random_angles)
        
        # Condições do modo
        r.append(tr[i]["v"] == V_HIGH)
        r.append(tr[i]["a"] == random_angles[0])
        
        # Condições do estado
        r.append(tr[i]["x"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["y"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["t"] == 0)
        
    r = And(r)
    
    return r


def switch_safe(tr, boat_id):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(tr)):
        if i != boat_id:
            # Condição das distâncias
            x = Abs(tr[i]["x"]-tr[boat_id]["x"]) > SAFE_DIST
            y = Abs(tr[i]["y"]-tr[boat_id]["y"]) > SAFE_DIST
            
            # Condição do tempo
            v_med = (tr[i]["v"] + tr[boat_id]["v"]) / 2
            t = Abs(tr[i]["t"]-tr[boat_id]["t"]) * v_med > SAFE_DIST
            
            # Verificar se uma das distâncias é superior à distância de segurança
            r.append(Or(x, y, t))
            
    r = And(r)
            
    return r
        
    
def timed(prev, curr):
    routes = ALL_ROUTES
    
    # Iterar cada um dos barcos
    r = []
    for i in range(len(prev)):
        
        # Condição dos modos
        r.append(curr[i]["v"] == prev[i]["v"])
        r.append(curr[i]["a"] == prev[i]["a"])
        
        # Condição do tempo
        r.append(curr[i]["t"] > prev[i]["t"])
        
        # Condição da rota e posição
        routes_cond = []
        for j in range(len(routes)):
            route_cond = []
            
            # Condição do ângulo
            route = routes[j]
            route_cond.append(prev[i]["a"] == route)
            
            # Incremento de posição
            dx = prev[i]["v"] * cos(deg_to_rad(route)) * (curr[i]["t"] - prev[i]["t"])
            dy = prev[i]["v"] * sin(deg_to_rad(route)) * (curr[i]["t"] - prev[i]["t"])
            
            # Condição da posição
            route_cond.append(curr[i]["x"] == prev[i]["x"] + dx)
            route_cond.append(curr[i]["y"] == prev[i]["y"] + dy)
            
            # Fazer o And de todas as condições
            routes_cond.append(And(route_cond))
        
        # Adicionar uma de todas as rotas possíveis
        r.append(Or(routes_cond))
        
    r = And(r)
        
    return r


def untimed(prev, curr):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(prev)):
        
        # Condições da posição e tempo
        r.append(curr[i]["x"] == prev[i]["x"])
        r.append(curr[i]["y"] == prev[i]["y"])
        r.append(curr[i]["t"] == prev[i]["t"])

        # barco V_LOW transita para V_LOW
        low_low = []
        low_low.append(And(prev[i]["v"] == V_LOW, curr[i]["v"] == V_LOW))
        low_low.append(Not(switch_safe(prev, i)))
        low_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        low_low = And(low_low)

        # barco V_LOW transita para barco V_HIGH
        low_high = []
        low_high.append(And(prev[i]["v"] == V_LOW, curr[i]["v"] == V_HIGH))
        low_high.append(switch_safe(prev, i))
        low_high.append(curr[i]["a"] == prev[i]["a"])
        low_high = And(low_high)

        # barco V_HIGH transita para barco V_LOW
        high_low = []
        high_low.append(And(prev[i]["v"] == V_HIGH, curr[i]["v"] == V_LOW))
        high_low.append(Not(switch_safe(prev, i)))
        high_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        high_low = And(high_low)

        # barco V_HIGH transita para barco V_HIGH
        high_high = []
        high_high.append(And(prev[i]["v"] == V_HIGH, curr[i]["v"] == V_HIGH))
        high_high.append(switch_safe(prev, i))
        high_high.append(curr[i]["a"] == prev[i]["a"])
        high_high = And(high_high)

        # Adicionar uma destas possíveis transições
        r.append(Or(low_low, low_high, high_low, high_high))
        
    # O estado não pode ficar igual
    same = []
    for i in range(len(prev)):
        same.append(prev[i]["v"] == curr[i]["v"])
        same.append(prev[i]["a"] == curr[i]["a"])
        same.append(prev[i]["x"] == curr[i]["x"])
        same.append(prev[i]["y"] == curr[i]["y"])
        same.append(prev[i]["t"] == curr[i]["t"])
    same = Not(And(same))

    # Todas as condições da transição devem ser cumpridas
    r = And(And(r), same)
    
    return r


def trans(prev, curr):
    # Condições timed e untimed
    untimed_cond = untimed(prev, curr)
    timed_cond = timed(prev, curr)
    
    # Condições de sincronismo
    eq_cond = And([curr[i]["t"] == curr[i+1]["t"] for i in range(len(curr)-1)])
    
    r = And(Or(untimed_cond, timed_cond), eq_cond)
    
    return r

In [7]:
def gen_trace(declare, init, trans, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i], trace[i+1]))
        
    if solver.check() == sat:
        m = solver.model()
        
        for i in range(k):
            print(f"\nTrace {i}:")
            for b in range(len(trace[i])):
                for v in trace[i][b]:
                    if trace[i][b][v].sort() == RealSort():
                        print(f"  {v}{b} = {z3tofloat(m[trace[i][b][v]])}")
                    else:
                        print(f"  {v}{b} = {m[trace[i][b][v]]}")
                print("")
            print("")
                
        r = m
    else:
        r = None
        
    return r

gen_trace(declare, init, trans, 10)


Trace 0:
  v0 = 10
  a0 = 75
  x0 = 49.6176414222108
  y0 = 67.88744840373758
  t0 = 0.0

  v1 = 10
  a1 = 240
  x1 = -96.75802346490728
  y1 = -146.12562400731912
  t1 = 0.0

  v2 = 10
  a2 = 240
  x2 = -91.27035159134527
  y2 = 56.46313064082268
  t2 = 0.0



Trace 1:
  v0 = 10
  a0 = 75
  x0 = 49.6176414222108
  y0 = 67.88744840373758
  t0 = 1.0

  v1 = 10
  a1 = 240
  x1 = -96.75802346490728
  y1 = -146.12562400731912
  t1 = 1.0

  v2 = 10
  a2 = 240
  x2 = -91.27035159134527
  y2 = 56.46313064082268
  t2 = 1.0



Trace 2:
  v0 = 10
  a0 = 75
  x0 = 49.6176414222108
  y0 = 67.88744840373758
  t0 = 2.0

  v1 = 10
  a1 = 240
  x1 = -96.75802346490728
  y1 = -146.12562400731912
  t1 = 2.0

  v2 = 10
  a2 = 240
  x2 = -91.27035159134527
  y2 = 56.46313064082268
  t2 = 2.0



Trace 3:
  v0 = 10
  a0 = 75
  x0 = 49.6176414222108
  y0 = 67.88744840373758
  t0 = 3.0

  v1 = 10
  a1 = 240
  x1 = -96.75802346490728
  y1 = -146.12562400731912
  t1 = 3.0

  v2 = 10
  a2 = 240
  x2 = -91.27035

[b1_x5 = -1209475293311341/12500000000000,
 b2_a4 = 240,
 b0_a5 = 75,
 b2_v4 = 10,
 b1_y6 = -1826570300091489/12500000000000,
 b0_y3 = 3394372420186879/50000000000000,
 b2_y4 = 1411578266020567/25000000000000,
 b2_x6 = -9127035159134529/100000000000000,
 b2_y8 = 1411578266020567/25000000000000,
 b2_t1 = 1,
 b1_v7 = 10,
 b1_a4 = 240,
 b1_x6 = -1209475293311341/12500000000000,
 b1_a7 = 240,
 b2_x4 = -9127035159134529/100000000000000,
 b0_a6 = 75,
 b0_y8 = 3394372420186879/50000000000000,
 b0_x9 = 9923528284442159/200000000000000,
 b2_x5 = -9127035159134529/100000000000000,
 b1_a5 = 240,
 b1_x4 = -1209475293311341/12500000000000,
 b2_v8 = 10,
 b0_a4 = 75,
 b0_v9 = 10,
 b2_x7 = -9127035159134529/100000000000000,
 b0_v8 = 10,
 b2_t9 = 89/10,
 b2_t4 = 4,
 b2_a5 = 240,
 b1_x8 = -1209475293311341/12500000000000,
 b2_t2 = 2,
 b0_x4 = 9923528284442159/200000000000000,
 b2_v9 = 10,
 b1_y5 = -1826570300091489/12500000000000,
 b2_a8 = 240,
 b2_x9 = -9127035159134529/100000000000000,
 b0_a9 = 75,
 b2_v5 = 10,
 b0_y5 = 3394372420186879/50000000000000,
 b0_a3 = 75,
 b2_v6 = 10,
 b1_y8 = -1826570300091489/12500000000000,
 b1_y9 = -1826570300091489/12500000000000,
 b2_t5 = 5,
 b2_a7 = 240,
 b2_a6 = 240,
 b0_x7 = 9923528284442159/200000000000000,
 b1_a6 = 240,
 b1_y7 = -1826570300091489/12500000000000,
 b0_x3 = 9923528284442159/200000000000000,
 b2_t7 = 69/10,
 b1_v4 = 10,
 b2_x8 = -9127035159134529/100000000000000,
 b1_v5 = 10,
 b0_y6 = 3394372420186879/50000000000000,
 b1_x9 = -1209475293311341/12500000000000,
 b1_v9 = 10,
 b2_a9 = 240,
 b2_t8 = 79/10,
 b0_v4 = 10,
 b1_v6 = 10,
 b2_y9 = 1411578266020567/25000000000000,
 b0_y9 = 3394372420186879/50000000000000,
 b0_v3 = 10,
 b0_y7 = 3394372420186879/50000000000000,
 b1_y4 = -1826570300091489/12500000000000,
 b2_y7 = 1411578266020567/25000000000000,
 b0_x6 = 9923528284442159/200000000000000,
 b0_v7 = 10,
 b1_x7 = -1209475293311341/12500000000000,
 b2_t6 = 6,
 b1_a9 = 240,
 b1_a8 = 240,
 b1_v8 = 10,
 b2_t3 = 3,
 b0_a7 = 75,
 b0_x8 = 9923528284442159/200000000000000,
 b0_x5 = 9923528284442159/200000000000000,
 b0_v5 = 10,
 b2_y6 = 1411578266020567/25000000000000,
 b0_v6 = 10,
 b2_v7 = 10,
 b0_a8 = 75,
 b0_y4 = 3394372420186879/50000000000000,
 b2_y5 = 1411578266020567/25000000000000,
 b0_a2 = 75,
 b2_y3 = 1411578266020567/25000000000000,
 b2_x3 = -9127035159134529/100000000000000,
 b2_a3 = 240,
 b1_a3 = 240,
 b2_v3 = 10,
 b1_v3 = 10,
 b1_y3 = -1826570300091489/12500000000000,
 b0_v2 = 10,
 b1_x3 = -1209475293311341/12500000000000,
 b2_y2 = 1411578266020567/25000000000000,
 b2_x2 = -9127035159134529/100000000000000,
 b2_a2 = 240,
 b1_a2 = 240,
 b2_v2 = 10,
 b1_v2 = 10,
 b1_y2 = -1826570300091489/12500000000000,
 b1_x2 = -1209475293311341/12500000000000,
 b0_y2 = 3394372420186879/50000000000000,
 b2_y1 = 1411578266020567/25000000000000,
 b2_x1 = -9127035159134529/100000000000000,
 b2_a1 = 240,
 b2_v1 = 10,
 b1_y1 = -1826570300091489/12500000000000,
 b1_x1 = -1209475293311341/12500000000000,
 b1_a1 = 240,
 b1_v1 = 10,
 b0_y1 = 3394372420186879/50000000000000,
 b0_x1 = 9923528284442159/200000000000000,
 b0_a1 = 75,
 b0_t9 = 89/10,
 b1_t9 = 89/10,
 b0_t8 = 79/10,
 b1_t8 = 79/10,
 b0_t7 = 69/10,
 b1_t7 = 69/10,
 b0_t6 = 6,
 b1_t6 = 6,
 b0_t5 = 5,
 b1_t5 = 5,
 b0_t4 = 4,
 b1_t4 = 4,
 b0_t3 = 3,
 b1_t3 = 3,
 ...]

Para determinar as possibilidades pode ser mais fácil verificar antes a velocidade de cada barco.

Se a velocidade é low e passa a high, então o barco no traço anterior está a salvo

Se a velocidade é low e mantém-se a low, então o barco no traço anterior está em perigo

Se a velocidade é high e mantém-se a high, então o barco no traço anterior está a salvo

Se a velocidade é high e passa a low, então o barco no traço anterior está em perigo